# KEN3140 (Semantic Web) Assignment 3
### Constructing ontologies with the OWL API

**Date:** 1 October 2020

**Author:** Kody Moodley

**License:** [GNU Lesser General Public License v3 (LGPL-3.0)](https://tldrlegal.com/license/gnu-lesser-general-public-license-v3-(lgpl-3))

**Assignment task overview:**

This assignment will assess your ability to construct an OWL 2 DL ontology that captures knowledge about a domain of interest - universities (or academic institutions). You will build your ontology using a similar Jupyter notebook to the one you used for Lab 9.

**Instructions: please read this section carefully before beginning the assignment.**

For detailed instructions, please consult ``Assignment 3 description.pdf``

**Notebook structure**

* Sections 1 - 3 of this notebook are purely for setting up the wrapper functions to allow you to build ontologies more concisely and quickly
* Section 4 provides some example commands to show you how to construct classes, roles, individuals and axioms using most of the OWL 2 DL features
* Section 4 also shows how to load, save to file, and visualise your ontologies
* Section 5 demonstrates how to execute reasoning tasks on your ontology
* You can reuse (copy and paste) code from Sections 4 and 5 if you wish in order to complete the assignment
* For From Section 6 onwards is where you will actually construct your own ontology using commands such as those in Sections 4 and 5

**Learning objectives:**

* How to construct an OWL 2 DL ontology programmatically (and to transfer what you have learned in this process within the “family relations” domain to another domain).
* To assess which OWL features are suited to capture certain kinds of knowledge about a domain. I.e., to accurately capture knowledge about the domain using OWL semantics
* To be able to judge whether two syntactically different axioms or class expressions represent the same knowledge (i.e., are semantically equivalent)
* How to apply an ontology design pattern in the construction of ontologies
* To be able to distinguish between classes, instances and object properties

**Technical requirements:**

* Jupyter notebooks: **https://jupyter.org/**
* IJava: **https://github.com/SpencerPark/IJava** (Java kernel for Jupyter notebooks)

**Documentation:**

For more details about how to use the OWL API, consult the documentation on Github: **https://github.com/owlcs/owlapi/wiki/Documentation**
The API (Java) docs for OWL API version 4 are located here: **http://owlcs.github.io/owlapi/apidocs_4/** and those for OWL API version 5 are located here: **http://owlcs.github.io/owlapi/apidocs_5/**
This notebook makes use of OWL API version 5 (specifically 5.1.2)

### 1. Import the OWL API &  OWL Reasoner
First we need to import the OWL API Java libraries and dependencies for constructing and editing ontologies,and an OWL reasoner for reasoning with these ontologies. The OWL API is maintained by, amongst others, [Ignazio Palmisano](https://github.com/ignazio1977).

In [ ]:
%jars ../bin/owlapi/lib/*.jar

In [ ]:
%jars ../bin/hermit/*.jar

In [ ]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.*;
import uk.ac.manchester.cs.owl.owlapi.*;
import org.semanticweb.owlapi.expression.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.util.mansyntax.ManchesterOWLSyntaxParser;
import org.semanticweb.owlapi.manchestersyntax.parser.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.formats.*;
import java.io.FileOutputStream;
import java.util.stream.*;
import org.semanticweb.owlapi.model.parameters.*;
import org.semanticweb.HermiT.*;
import org.semanticweb.owlapi.reasoner.*;
import com.clarkparsia.owlapi.explanation.DefaultExplanationGenerator;
import com.clarkparsia.owlapi.explanation.util.*;

### 2. Create a fresh ontology

In [ ]:
// Ontology manager
OWLOntologyManager manager=OWLManager.createOWLOntologyManager();
// Ontology IRI
IRI ontologyIRI = IRI.create("http://maastrichtuniversity.nl/semweb/2020/lab9#");
// Create a fresh ontology
OWLOntology rootOntology=manager.createOntology(ontologyIRI);
// Create a data factory (ontology builder class)
OWLDataFactory df = new OWLDataFactoryImpl();
// entity renderer (prints ontology entities without IRI prefixes)
ManchesterOWLSyntaxOWLObjectRendererImpl m = new ManchesterOWLSyntaxOWLObjectRendererImpl();
// Initialise reasoner
OWLReasonerFactory rf = new Reasoner.ReasonerFactory();
OWLReasoner reasoner =  rf.createReasoner(rootOntology, new Configuration());

### 3. Wrapper functions & setup
Wrapper functions and classes to make it easier, more concise, and quicker, to use the OWL API to build ontologies. **There is no need to edit any code in this section**. If you are an experienced developer and are curious to tinker with this, that's fine - but then **we cannot provide any support or help if anything goes wrong**.

In [ ]:
// Mapping human readable resource names to full IRIs
class Provider extends CachingBidirectionalShortFormProvider {

    private SimpleShortFormProvider provider = new SimpleShortFormProvider();

    @Override
    protected String generateShortForm(OWLEntity entity) {
        return provider.getShortForm(entity);
    }
}

OWLDataFactory df = OWLManager.getOWLDataFactory();
Provider shortFormProvider = new Provider();
OWLEntityChecker entityChecker = new ShortFormEntityChecker(shortFormProvider);
ManchesterOWLSyntaxParser parser = OWLManager.createManchesterParser();
parser.setOWLEntityChecker(entityChecker);

// Creating entities:
// Create class
public void createClass(String classname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLClass c = df.getOWLClass(ontologyIRI.toString() + classname);
    shortFormProvider.add(c);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(c, new HashSet<OWLAnnotation>());
    OWLAxiom class_declaration = a.getAxiomWithoutAnnotations();
    manager.addAxiom(rootOntology, class_declaration);
}
                 
// Create role
public void createRole(String rolename, int trans, int ref, int sym) throws OWLOntologyCreationException, OWLOntologyStorageException {
    if (trans == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLTransitiveObjectPropertyAxiom t = df.getOWLTransitiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (ref == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLReflexiveObjectPropertyAxiom t = df.getOWLReflexiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (ref == 2){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLIrreflexiveObjectPropertyAxiom t = df.getOWLIrreflexiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (sym == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLSymmetricObjectPropertyAxiom t = df.getOWLSymmetricObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (sym == 2){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLAsymmetricObjectPropertyAxiom t = df.getOWLAsymmetricObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if ((trans == 0) && (ref == 0) && (sym == 0)) {
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, role_declaration);
    }
}

// Create individual
public void createIndividual(String individualname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLNamedIndividual i = df.getOWLNamedIndividual(ontologyIRI.toString() + individualname);
    shortFormProvider.add(i);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(i, new HashSet<OWLAnnotation>());
    OWLAxiom individual_declaration = a.getAxiomWithoutAnnotations();
    manager.addAxiom(rootOntology, individual_declaration);
}

// Create axiom
public OWLAxiom createAxiom(String axiomStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    parser.setStringToParse(axiomStr);
    OWLAxiom axiom = parser.parseAxiom();
    manager.addAxiom(rootOntology, axiom);
    return axiom;
}

// Create class expression
public OWLClassExpression createClassExpression(String classExpressionStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    parser.setStringToParse(classExpressionStr);
    OWLClassExpression clsEx = parser.parseClassExpression();
    return clsEx;
}

// Create role assertion axiom
public OWLAxiom createRoleAssertion(String axiomStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    String [] parts = axiomStr.split(" ");
    if (parts.length < 3){
        System.out.println("Parser error: incorrect syntax for role assertion. requires exactly three tokens separated by single spaces");
        return null;
    }
    else{
        OWLObjectPropertyAssertionAxiom a = df.getOWLObjectPropertyAssertionAxiom(df.getOWLObjectProperty(ontologyIRI.toString() + parts[1]), df.getOWLNamedIndividual(ontologyIRI.toString() + parts[0]), df.getOWLNamedIndividual(ontologyIRI.toString() + parts[2]));
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

// Make all individual names in the ontology refer to different objects in the domain
public OWLAxiom allIndividualsDifferent(){
    Set<OWLIndividual> inds = new HashSet<OWLIndividual>();
    rootOntology.individualsInSignature(Imports.EXCLUDED).forEach(c->inds.add(c));
    
    if (inds.size() < 2){
        System.out.println("Error: requires more than 1 individual name in the ontology.");
        return null;
    }
    else{
        OWLAxiom a = df.getOWLDifferentIndividualsAxiom(inds, new HashSet<OWLAnnotation>()); 
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

// Make **specified** individual names in the ontology refer to different objects in the domain
public OWLAxiom different(String differentIndividualsStr){
    String [] parts = differentIndividualsStr.split(" ");
    Set<OWLIndividual> inds = new HashSet<OWLIndividual>();
    for (String s: parts){
        inds.add(df.getOWLNamedIndividual(IRI.create(ontologyIRI.toString() + s)));
    }
    
    if (parts.length < 2){
        System.out.println("Parser error: incorrect syntax for different individuals axiom. requires more than 1 token (individual name) separated by single spaces");
        return null;
    }
    else{
        OWLAxiom a = df.getOWLDifferentIndividualsAxiom(inds, new HashSet<OWLAnnotation>()); 
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

// Reasoning:
// Get all atomic subclasses for the given class expression (both inferred by the reasoner and asserted)
public void getSubClasses(String classEx) throws OWLOntologyCreationException, OWLOntologyStorageException {
    reasoner.flush();
    NodeSet<OWLClass> subclasses = reasoner.getSubClasses(createClassExpression(classEx));
    for (Node<OWLClass> nc: subclasses){
        for (OWLClass c: nc){
            if (!c.isOWLNothing() && !c.isOWLThing())
                System.out.println(m.render(c));
        }
    }
}

// Get all atomic superclasses for the given class expression (both inferred by the reasoner and asserted)
public void getSuperClasses(String classEx) throws OWLOntologyCreationException, OWLOntologyStorageException {
    reasoner.flush();
    NodeSet<OWLClass> supclasses = reasoner.getSuperClasses(createClassExpression(classEx));
    for (Node<OWLClass> nc: supclasses){
        for (OWLClass c: nc){
            if (!c.isOWLNothing() && !c.isOWLThing())
                System.out.println(m.render(c));
        }
    }
}

// Get all unsatisfiable class names in the ontology
public void getUnsatisfiableClasses(){
    reasoner.flush();
    Node<OWLClass> classes = reasoner.getUnsatisfiableClasses();
    for (OWLClass c: classes){
        if (!c.isOWLNothing())
            System.out.println(m.render(c));
    }
}

// Check if the ontology has a model
public void isConsistent(){
    reasoner.flush();
    if (reasoner.isConsistent())
        System.out.println("Yes");
    else
        System.out.println("No");
}

// Get types for specific individual
public void getTypes(String ind){
    reasoner.flush();
    NodeSet<OWLClass> typesC = reasoner.getTypes(df.getOWLNamedIndividual(IRI.create(ontologyIRI.toString()+ind)));
    for (Node<OWLClass> c: typesC){
        for (OWLClass c2: c){
            if (!c2.isOWLThing())
                System.out.println(m.render(c2));
        }
    }
}

// Get types for **ALL** individuals in ontology
public void getAllTypes(){
    reasoner.flush();
    
    for (OWLIndividual i : rootOntology.getIndividualsInSignature(true)){
        System.out.println(m.render(i));
        System.out.println("-----------");
        getTypes(m.render(i));
        System.out.println();
    }
}

// Get property assertions for this property 
public void getRelations(String roleStr){
    reasoner.flush();
    System.out.println(roleStr);
    System.out.println("-----------");
    Set<OWLNamedIndividual> inds = rootOntology.getIndividualsInSignature(true);
    for (OWLNamedIndividual i :inds){
        NodeSet<OWLNamedIndividual> indP = reasoner.getObjectPropertyValues(i, df.getOWLObjectProperty(IRI.create(ontologyIRI.toString()+roleStr)));
        for (Node<OWLNamedIndividual> n: indP) {
            for (OWLNamedIndividual ai : n){
                System.out.println(m.render(i) + "," + m.render(ai));
            }
        }        
    }

    System.out.println();
}

// Get property assertions **ALL** properties in ontology
public void getAllRelations(){
    reasoner.flush();
    for (OWLObjectProperty o : rootOntology.getObjectPropertiesInSignature(true)){
        getRelations(m.render(o));
        System.out.println();
    }
}

// True if axiom follows, false otherwise
public void isEntailed(String axiomStr){
    parser.setStringToParse(axiomStr);
    OWLAxiom axiom = parser.parseAxiom();
    reasoner.flush();
    if (reasoner.isEntailed(axiom))
        System.out.println("Yes");
    else
        System.out.println("No");
}

// Explain unsatisfiability
public void explainUnsatisfiability(String clsStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    DefaultExplanationGenerator d = new DefaultExplanationGenerator(manager, rf, rootOntology, new SilentExplanationProgressMonitor());
    OWLClassExpression cls = createClassExpression(clsStr);
    Set<Set<OWLAxiom>> explanations = d.getExplanations(cls);

    int count = 1;
    for (Set<OWLAxiom> exp: explanations){
        System.out.println("Explanation " + count);
        System.out.println("--------------");
        for (OWLAxiom ex: exp){
            System.out.println(m.render(ex));
        }
        count++;
        System.out.println();
    }
}

// Explain entailment
public void explainEntailment(String axiomStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    DefaultExplanationGenerator d = new DefaultExplanationGenerator(manager, rf, rootOntology, new SilentExplanationProgressMonitor());
    parser.setStringToParse(axiomStr);
    OWLAxiom axiom = parser.parseAxiom();
    Set<Set<OWLAxiom>> explanations = d.getExplanations(axiom);

    int count = 1;
    for (Set<OWLAxiom> exp: explanations){
        System.out.println("Explanation " + count);
        System.out.println("--------------");
        for (OWLAxiom ex: exp){
            System.out.println(m.render(ex));
        }
        count++;
        System.out.println();
    }
}

// Printing:
// print ontology to output
public void print(OWLOntology ontology){
    Stream<OWLEntity> signature = ontology.signature(Imports.EXCLUDED);
    Stream<OWLAxiom> rbox = ontology.rboxAxioms(Imports.EXCLUDED);
    Stream<OWLAxiom> tbox = ontology.tboxAxioms(Imports.EXCLUDED);
    Stream<OWLAxiom> abox = ontology.aboxAxioms(Imports.EXCLUDED);
    
    Set<OWLEntity> clses = new HashSet<OWLEntity>();
    Set<OWLEntity> roles = new HashSet<OWLEntity>();
    Set<OWLEntity> inds = new HashSet<OWLEntity>();
    signature.forEach(e -> {
        if (e.isOWLClass()) clses.add(e);
        if (e.isOWLObjectProperty()) roles.add(e);
        if (e.isIndividual()) inds.add(e);
    });
    
    System.out.println();
    
    System.out.println("Classes:");
    System.out.println("--------");
    
    for (OWLEntity c: clses){
        System.out.println(m.render(c));
    }
    
    System.out.println();
    
    System.out.println("Roles:");
    System.out.println("------");

    for (OWLEntity r: roles){
        System.out.println(m.render(r));
    }
    
    System.out.println();
    
    System.out.println("Individuals:");
    System.out.println("------------");
    
    for (OWLEntity i: inds){
        System.out.println(m.render(i));
    }

    System.out.println();

    System.out.println("TBox:");
    System.out.println("-----");

    tbox.forEach(a -> System.out.println(m.render(a)));
    
    System.out.println();
    
    System.out.println("ABox:");
    System.out.println("-----");

    abox.forEach(a -> System.out.println(m.render(a)));    
    
    System.out.println();

    System.out.println("RBox:");
    System.out.println("-----");
    
    rbox.forEach(a -> System.out.println(m.render(a)));  
}

// print an axiom to output
public void print(OWLAxiom axiom){
    System.out.println(m.render(axiom));
}

// print some statistics about the ontology
public void printStatistics(OWLOntology ontology){
    System.out.println();
    System.out.println("Ontology statistics:");
    System.out.println("--------------------");
    // Number of axioms and constructs in ontology
    System.out.println("Number of axioms: " + ontology.getAxiomCount());
    System.out.println("Number of classes: " + ontology.getClassesInSignature(true).size());
    System.out.println("Number of relations: " + ontology.getObjectPropertiesInSignature(true).size());
    System.out.println("Number of instances: " + ontology.getIndividualsInSignature(true).size());
    // Number of axioms of a specific type in ontology
    System.out.println("Number of SubClassOf axioms: " + ontology.getAxioms(AxiomType.SUBCLASS_OF).size());
    System.out.println("Number of EquivalentClasses axioms: " + ontology.getAxioms(AxiomType.EQUIVALENT_CLASSES).size());
    System.out.println("Number of DisjointClasses axioms: " + ontology.getAxioms(AxiomType.DISJOINT_CLASSES).size());
    System.out.println("Number of Class assertions: " + ontology.getAxioms(AxiomType.CLASS_ASSERTION).size());
    System.out.println("Number of ObjectProperty assertions: " + ontology.getAxioms(AxiomType.OBJECT_PROPERTY_ASSERTION).size());
}

// Removing entities:
// remove atomic class
public void removeClass(String classname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLClass c = df.getOWLClass(ontologyIRI.toString() + classname);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(c, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

// remove atomic role
public void removeRole(String rolename) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

// remove individual
public void removeIndividual(String individualname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLNamedIndividual i = df.getOWLNamedIndividual(ontologyIRI.toString() + individualname);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(i, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

// remove axiom
public void removeAxiom(OWLAxiom a) throws OWLOntologyCreationException, OWLOntologyStorageException {
    manager.removeAxiom(rootOntology, a);
}

// remove all axioms from ontology
public void resetOntology(){
    rootOntology.removeAxioms(rootOntology.getAxioms());
}

// save ontology to file (given filepath string)
public void saveOntology(String filepath) throws OWLOntologyStorageException, FileNotFoundException{
    FileOutputStream fout=new FileOutputStream(filepath);    
    manager.saveOntology(rootOntology, new ManchesterSyntaxDocumentFormat(), fout);
}

### 4. Examples to get started
Have a look at these examples to show you how to build ontologies using this "wrapper" API I created for the OWL API

#### 4a. Creating (atomic) classes

In [ ]:
// create class **names**
// You can only create **atomic** classes with the method below.
// For complex class expressions, you can create axioms (i.e. using the createAxiom method) 
// that defines or describes the relationship of this expression with others in the ontology.
// E.g. (Person and (hasChild some (hasChild some Person ))) defines "grandparents"
// But you would not just add this expression by itself to the ontology
// Generally, you want to express a logical statement using this expression:
// E.g. Parent equivalentTo (Person and (hasChild some (hasChild some Person )))
createClass("Woman");
createClass("Man");
createClass("Male");
createClass("Female");
createClass("Person");
createClass("Adult");
createClass("Car");
createClass("Tyre");
createClass("USCitizen");
createClass("Passport");
createClass("Leg");
createClass("Arthropod");
createClass("JohnsBookClubMembers");
print(rootOntology);

#### 4b. Creating (atomic) roles

In [ ]:
// create roles
// parameter1: name of the role
// parameter2: transitive? 0 - no, 1 - yes
// parameter3: reflexive? 0 - no, 1 - yes, 2 - irreflexive
// parameter4: symmetric? 0 - no, 1 - yes, 2 - antisymmetric
createRole("hasGender", 0, 0, 0);
createRole("marriedTo", 0, 2, 1);
createRole("hasPart", 0, 0, 0);
createRole("teaches", 0, 0, 0);
createRole("studentOf", 0, 0, 0);
createRole("owns", 0, 0, 0);
createRole("hasSister", 0, 0, 0);
createRole("hasSibling", 0, 0, 0);
createRole("hasDocument", 0, 0, 0);
print(rootOntology);

#### 4c. Creating individuals

In [ ]:
// create individuals
createIndividual("john");
createIndividual("sara");
createIndividual("ford_mondeo_16");
createIndividual("mary");
print(rootOntology);

#### 4d. Creating TBox axioms

In [ ]:
// TBox axioms
// create subclass axioms
OWLAxiom subAx1 = createAxiom("Man subClassOf Person and (hasGender some Male)");
OWLAxiom subAx2 = createAxiom("Woman subClassOf Person and (hasGender some Female)");
// create equivalence axioms
OWLAxiom eqAx1 = createAxiom("Man equivalentTo Person and Adult and (hasGender some Male)");
OWLAxiom eqAx2 = createAxiom("Woman equivalentTo Person and Adult and (hasGender some Female)");
// create disjointness axioms
OWLAxiom disjAx1 = createAxiom("Man disjointWith Woman");
OWLAxiom disjAx2 = createAxiom("Female disjointWith Male");
// create axioms using cardinality restrictions
OWLAxiom cardAx1 = createAxiom("Car subClassOf hasPart exactly 4 Tyre");
OWLAxiom cardAx2 = createAxiom("USCitizen subClassOf hasDocument max 2 Passport");
OWLAxiom cardAx3 = createAxiom("Arthropod subClassOf hasPart min 6 Leg");
// create axioms using nominals
OWLAxiom nAx1 = createAxiom("JohnsBookClubMembers equivalentTo {john,mary,sara}");
//print(rootOntology);
// create domain & range axioms
OWLAxiom nAx1 = createAxiom("hasGender Domain: Person");
OWLAxiom nAx1 = createAxiom("hasGender Range: Male or Female");
print(rootOntology);

//System.out.println(reasoner.isConsistent());

#### 4e. Creating ABox axioms

In [ ]:
// ABox axioms
// create class assertion
OWLAxiom caAx1 = createAxiom("john Type: Adult");
OWLAxiom caAx2 = createAxiom("mary Type: Adult");
// **NB: create role assertion**
// With role assertions the method name is different
// **we cannot use createAxiom here**
// We use createRoleAssertion instead
OWLAxiom raAx1 = createRoleAssertion("john hasGender Male");
OWLAxiom raAx2 = createRoleAssertion("mary hasGender Female");
OWLAxiom raAx3 = createRoleAssertion("john marriedTo mary");
// Create different individuals axiom (specify each individual name)
//OWLAxiom diAx1 = createAxiom("DifferentIndividuals: john mary sara");
OWLAxiom diAx1 = different("john mary sara");
// Make **all** individual names in the ontology refer to distinct objects from the domain
//OWLAxiom allDiff = allIndividualsDifferent();
print(rootOntology);

#### 4f. Creating RBox axioms

In [ ]:
// RBox axioms
// **note the ":" after SubPropertyOf and InverseOf**
// role hierarchies 
OWLAxiom roleHAx1 = createAxiom("hasSister SubPropertyOf: hasSibling");
// role compositions
OWLAxiom roleCAx1 = createAxiom("owns o hasPart SubPropertyOf: owns");
// inverse roles
OWLAxiom invAx1 = createAxiom("teaches InverseOf: studentOf");
print(rootOntology);
printStatistics(rootOntology);

#### 4g. Saving ontology to file

In [ ]:
// // Save the ontology
// saveOntology("ontologyfilename.owl");
// // If you want to "reset" the ontology
// // I.e., remove all axioms from the ontology
// // Use the following command to start afresh
// resetOntology();
// // print the ontology
// print(rootOntology);

#### 4h. What if I want to load an ontology from a local file?

In [ ]:
// // Obtain a reference to a local OWL ontology file
// File file = new File("ontologyfilename.owl");
// // **NB:** you can't store two ontologies with the same IRI in the same ontology manager instance
// // Create a new ontology manager
// OWLOntologyManager newManager = OWLManager.createOWLOntologyManager();
// // Load the ontology from file into the **new** ontology manager
// OWLOntology another_ontology = newManager.loadOntologyFromOntologyDocument(file);
// print(another_ontology);
// // some statistics about the ontology entities and axioms
// printStatistics(another_ontology);

#### 4i. ...or load one from a remote URI (hosted on the Web)

In [ ]:
// // Specify the IRI where the ontology is located
// IRI remoteOntologyIRI = IRI.create("https://protege.stanford.edu/ontologies/pizza/pizza.owl");
// //IRI remoteOntologyIRI = IRI.create("https://web.archive.org/web/20111213110713/http://www.movieontology.org/2010/01/movieontology.owl");
// // Load the ontology from the URI into the ontology manager
// OWLOntology remote_ontology = manager.loadOntology(remoteOntologyIRI);
// print(remote_ontology);
// printStatistics(remote_ontology);

#### 4j. Visualise your ontology using [WebVowel](http://www.visualdataweb.de/webvowl/)
You can upload your OWL file (less than 5MB) to the WebVowel online ontology visualiser to explore your ontology graphically. Protege also has ontology visualisation plugins e.g. [OntoGraf](https://github.com/protegeproject/ontograf), you can find this under Window>Tabs>OntoGraf. However, WebVowel has a more visually appealing interface.

![title](img/visualise_your_ont.png)

#### 5. Reasoning with your ontology using OWL 2 DL reasoners
Here we will use [HermiT](http://www.hermit-reasoner.com/) which is a robust OWL 2 DL reasoner. There is also [Pellet](https://github.com/stardog-union/pellet) and [Fact++](https://bitbucket.org/dtsarkov/factplusplus/downloads/uk.ac.manchester.cs.owl.factplusplus-P5.x-v1.6.5.jar) as alternatives amongst [others](https://www.w3.org/2001/sw/wiki/OWL/Implementations)

#### 5a. Classify the ontology 
Computes all subclass / superclass relationships between all atomic concepts in the ontology

In [ ]:
// Classify the ontology
reasoner.precomputeInferences(InferenceType.CLASS_HIERARCHY);
// Get super classes of a given class expression / atomic class
getSuperClasses("hasGender some Male");
System.out.println();
// Get subclasses of a given class expression / atomic class
getSubClasses("hasGender some Male");

#### 5b. Is the ontology consistent?

In [ ]:
createClass("Bird");
createClass("Penguin");
createClass("FlyingOrganism");
createAxiom("Bird subClassOf FlyingOrganism");
createAxiom("Penguin subClassOf Bird");
createAxiom("Penguin subClassOf not FlyingOrganism");
createIndividual("tweety");
OWLAxiom tweetyAxiom = createAxiom("tweety Type: Penguin");
// Check if ontology is consistent or not
isConsistent();

#### 5c. Compute all unsatisfiable classes

In [ ]:
removeAxiom(tweetyAxiom);
isConsistent();
System.out.println();
// Compute all unsatisfiable atomic classes in the ontology
getUnsatisfiableClasses();

#### 5c. TBox subsumption

In [ ]:
// Check if some axiom logicall follows from the ontology
isEntailed("Man subClassOf Bird");

#### 5d. ABox realisation

In [ ]:
// Get all atomic classes which this individual belongs to
getTypes("mary");
System.out.println();
// Get all atomic classes for each individual in the ontology
getAllTypes();
System.out.println();
// Get all role assertions (triples) where the given object property is the subject of the assertion (triple)
getRelations("marriedTo");
System.out.println();
// Get all role assertions (inferred and asserted) in the ontology
getAllRelations();

#### 5e. Compute justifications

In [ ]:
// explain unsatisfiability
explainUnsatisfiability("Penguin");
// explain general entailment
explainEntailment("john Type: Person");

#### 6. Start of assignment task

In [ ]:
// Start building your ontology here //